## Geocoding and routing application using open source python libraries

##### The workflow order is as follows:
1. Import libraries
2. Import addresses
3. Geocode addresses
4. Plot addresses
5. Select start and end point to plot route
6. Compute and display optimal driving route, distance and time


In [1]:
import openrouteservice as ors
from openrouteservice.directions import directions
import folium
import geopy
import leafmap
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import io

In [2]:
#pip install openrouteservice

##### API Key

In [3]:
# import config
# performs requests to the ORS API services
# client will be used in all examples
client = ors.Client(key='5b3ce3597851110001cf624889ce32dc9e8741dca14bd53556e3a687')

#### Import addresses

Import addresses in CSV file format 

In [4]:
upload = widgets.FileUpload(
        accept='*.csv',
        multiple=False)
display(upload)

FileUpload(value={}, accept='*.csv', description='Upload')

#### Data Cleaning 
Inspect the imported data and optimize it for geocoding

In [6]:
input_file = list(upload.value.values())[0]
content = input_file['content']
content = io.StringIO(content.decode('utf-8'))
df = pd.read_csv(content)
df.head()

,Unnamed: 0,restaurant,street,zip,city,country
0,0,McDonalds,S-Bahnhof Alexanderplatz 1,10178,Berlin,Germany
1,1,McDonalds,U-Bahnhof Alexanderplatz 1,10178,Berlin,Germany
2,2,McDonalds,Europaplatz 1,10557,Berlin,Germany
3,3,McDonalds,Walther-Schreiber-Platz 1,12161,Berlin,Germany
4,4,McDonalds,Hansastr. 1,13053,Berlin,Germany


In [7]:
# Merge street, zip and city fields into an address field Using DataFrame.apply() and lambda function

df["address"] = df["street"].map(str) + ', ' + df["zip"].map(str)+ ', ' + df["city"]+ ', ' +  df['country']
df.head()

,Unnamed: 0,restaurant,street,zip,city,country,address
0,0,McDonalds,S-Bahnhof Alexanderplatz 1,10178,Berlin,Germany,"S-Bahnhof Alexanderplatz 1, 10178, Berlin, Ger..."
1,1,McDonalds,U-Bahnhof Alexanderplatz 1,10178,Berlin,Germany,"U-Bahnhof Alexanderplatz 1, 10178, Berlin, Ger..."
2,2,McDonalds,Europaplatz 1,10557,Berlin,Germany,"Europaplatz 1, 10557, Berlin, Germany"
3,3,McDonalds,Walther-Schreiber-Platz 1,12161,Berlin,Germany,"Walther-Schreiber-Platz 1, 12161, Berlin, Germany"
4,4,McDonalds,Hansastr. 1,13053,Berlin,Germany,"Hansastr. 1, 13053, Berlin, Germany"


#### Geocode addresses
Geocode addresses using geopy library and Nominatim

In [8]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [9]:
geolocator = Nominatim(user_agent="Practice")

In [10]:
locator = Nominatim(user_agent="Practice")

geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
df['location'] = df['address'].apply(geocode)

df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('S-Bahnhof Alexanderplatz 1, 10178, Berlin, Germany',), **{}).
Traceback (most recent call last):
  File "C:\Users\carellano.LEGION5PRO-O3OB\AppData\Roaming\Python\Python39\site-packages\urllib3\connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "C:\Users\carellano.LEGION5PRO-O3OB\AppData\Roaming\Python\Python39\site-packages\urllib3\util\connection.py", line 95, in create_connection
    raise err
  File "C:\Users\carellano.LEGION5PRO-O3OB\AppData\Roaming\Python\Python39\site-packages\urllib3\util\connection.py", line 85, in create_connection
    sock.connect(sa)
socket.timeout: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\carellano.LEGION5PRO-O3OB\AppData\Roaming\Python\Python39\site-packages\urllib3\connectionpool.py", line 703, in urlopen
    httplib_response = self._make_request(
  File "C:\Users

In [11]:
df.head()

,Unnamed: 0,restaurant,street,zip,city,country,address,location,point
0,0,McDonalds,S-Bahnhof Alexanderplatz 1,10178,Berlin,Germany,"S-Bahnhof Alexanderplatz 1, 10178, Berlin, Ger...",None,None
1,1,McDonalds,U-Bahnhof Alexanderplatz 1,10178,Berlin,Germany,"U-Bahnhof Alexanderplatz 1, 10178, Berlin, Ger...",None,None
2,2,McDonalds,Europaplatz 1,10557,Berlin,Germany,"Europaplatz 1, 10557, Berlin, Germany","(Europaplatz 1, 10557 Berlin, Europaplatz, Moa...","(52.5257446, 13.3685935, 0.0)"
3,3,McDonalds,Walther-Schreiber-Platz 1,12161,Berlin,Germany,"Walther-Schreiber-Platz 1, 12161, Berlin, Germany","(Schloss-Straßen-Center, 1, Walther-Schreiber-...","(52.46544755, 13.327684176306294, 0.0)"
4,4,McDonalds,Hansastr. 1,13053,Berlin,Germany,"Hansastr. 1, 13053, Berlin, Germany","(1, Hansastraße, Alt-Hohenschönhausen, Lichten...","(52.5470223, 13.4684079, 0.0)"


Drop any colummns with null coordinate values

In [ ]:
df1 = df.dropna()
df1.head()

In [ ]:
df1.shape

In [ ]:
# split point column into latitude, longitude and altitude columns
df1[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df1['point'].tolist(), index=df1.index)
df1.head()

In [ ]:
df1= df1.drop(['street', 'zip', 'city', 'point'], axis=1)

In [ ]:
df1.head()

#### Plot addresses
Plot geocoded addresses using folium and leafmap mapping libraries

In [ ]:
map1 = folium.Map(
    location=[50.1109,8.6821],
    tiles='openstreetmap',
    zoom_start=6,
)

df1.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]]).add_to(map1), axis=1)
map1

In [ ]:
Map = leafmap.Map(center=(50.1109,8.6821), zoom=5)
Map.add_xy_data(df1, x="longitude", y="latitude", layer_name="restaurants")
Map

#### Routing from start point to end point
Implement routing using openroutes service library


In [ ]:
df1.address

##### Select start point for routing
Specify a start point or origin from the dropdown list of geocoded addresses


In [ ]:
start_dropdown = widgets.Dropdown(options=list(df1.address),
                                     value=list(df1.address)[0],
                                     description='start:',
                                     disabled=False)
start_dropdown

##### Select end point for routing
Specify an end point or destination from the dropdown list of geocoded addresses


In [ ]:
end_dropdown = widgets.Dropdown(options=list(df1.address),
                                   value=list(df1.address)[-1],
                                   description='end:',
                                   disabled=False)
end_dropdown


# widgets
start_dropdown = widgets.Dropdown(options=list(df.address),
                                     value=list(df.address)[0],
                                     description='start:',
                                     disabled=False)

end_dropdown = widgets.Dropdown(options=list(df.address),
                                   value=list(df.address)[1],
                                   description='end:',
                                   disabled=False)

# function
# calculate distance from a to b
# display map with ant path from a to b
def get_coordinates(a, b):
    start_lat = df[df.address == a].latitude.values[0]
    start_long = df[df.address == a].longitude.values[0]
    end_lat = df[df.address == b].latitude.values[0]
    end_long = df[df.address == b].longitude.values[0]
    start, end = (start_lat, start_long), (end_lat, end_long)
    
    m = Map(center=(50.1109,8.6821), zoom=5, scroll_wheel_zoom=True)
    ant_path = AntPath(locations=[start, end])
    m.add_layer(ant_path)
    measure = MeasureControl(position='topleft', active_color='red', primary_length_unit='miles')
    m.add_control(measure)
    display(m)
    
    return distance.distance(start, end).miles

# interaction between function and widget
widgets.interact(get_coordinates, a=start_dropdown, b=end_dropdown);

In [ ]:
#store selected dropdown values
starter = start_dropdown.value
ender = end_dropdown.value

print('start_dropdown :',starter)
print('end_dropdown :',ender)

# # interaction between function and widget
# widgets.interact(get_coordinates, a=start_dropdown, b=end_dropdown);
def get_coordinates(starter, ender):
    start_lat = df1[df1.address == starter].latitude.values[0]
    start_long = df1[df1.address == starter].longitude.values[0]
    end_lat = df1[df1.address == ender].latitude.values[0]
    end_long = df1[df1.address == ender].longitude.values[0]
    start, end = ((start_long, start_lat), (end_long, end_lat))
    return start, end

# call fn and return start, end coordinates
start_and_end = get_coordinates(starter,ender)
print(start_and_end)

#reverse zoom center point, refered from start coordinates. should be [lat lon]
start_loc = list(start_and_end[0])
start_loc[1],start_loc[0] = start_loc[0], start_loc[1]
end_loc = list(start_and_end[1])
end_loc[1],end_loc[0] = end_loc[0], end_loc[1]


#store markers coordinates and labels
marks =[start_loc, end_loc]
marks_popup=[starter,ender]


# directions
route = client.directions(coordinates=start_and_end,
                          profile='driving-car', 
                          format='geojson')


def get_distance_and_time(route):
    level1 = route['features']
    level2 =list(level1)[0].get('properties')
    level3= level2.get('summary')
    #store distance traveled
    distance = level3.get('distance')
    #store trip duration
    duration = (level3.get('duration')/60)
    print('Distance Travelled : ', round(distance/1000,2), 'km')
    print('Trip duration : ', round(duration,2), 'mins')
    return distance, duration


#execute get_distance_time()
route_details = get_distance_and_time(route)
distance_traveled, time_traveled = route_details[0], route_details[1]


#location wants lat lon
# define empty map
map_directions = folium.Map(location=start_loc, zoom_start=12.5)

# add geojson of route to map
folium.GeoJson(route, name='route').add_to(map_directions)

# add layer control to map (allows layer to be turned on or off)
folium.LayerControl().add_to(map_directions)

# add markers on start and end
for i in range(len(marks)):
    folium.Marker(location=marks[i], popup=marks_popup[i]).add_to(map_directions)

# display map
map_directions